In [1]:
import os, sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
  sys.path.append(module_path)

from utils.utils import Create_Similarity_Model

2022-07-05 14:07:19.186041: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-05 14:07:19.186055: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Now we will create our model

In [2]:
n_nonsimilar = 1
n_similar = 1

### We will look for similarity in the last n_similar parameters of the inputs.

model = Create_Similarity_Model(n_nonsimilar = n_nonsimilar, n_similar = n_similar)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 similarity_layer_1 (Dense)     (None, 1)            1           ['lambda[0][0]']                 
                                                                                              

2022-07-05 14:07:21.828786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-05 14:07:21.829386: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-05 14:07:21.829466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-05 14:07:21.829540: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-05 14:07:21.829625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

## Define the loss, optimizer and metrics

In [3]:
loss = 'mean_squared_error'
optimizer = 'adam'
metrics = ['mean_squared_error']

model.compile(loss = loss, optimizer = optimizer, metrics = metrics)

## Fabricate our data simulating the laminar flow

In [7]:
possible_re_tau = np.linspace(start = 1, stop = 100, num = 100)
Re_tau = []
Y_ =[]
U_ = []
dic_x = {}
dic_y = {}

for re_tau in possible_re_tau:
    possible_y_= np.linspace(start = 0.1, stop = re_tau, num = 100)
    for y_ in possible_y_:
        Y_.append(y_)
        Re_tau.append(re_tau)
        u_ = y_ - ((1 / (2*re_tau)) * ((y_) ** 2))
        U_.append(u_)

Y_ = np.log(Y_)
Re_tau = np.log(Re_tau)

dic_x['y+'] = Y_
dic_x['re_tau'] = Re_tau
dic_y['u+'] = U_

nonsimilar_keys = ['y+']

xtrain = pd.DataFrame.from_dict(dic_x).values
ytrain = pd.DataFrame.from_dict(dic_y).values

<class 'numpy.ndarray'>
[[ 0.095     ]
 [ 0.1031405 ]
 [ 0.11119835]
 ...
 [49.97963471]
 [49.99490868]
 [50.        ]]


In [8]:
model.fit(xtrain, ytrain, epochs = 1000, verbose = 1)

Epoch 1/1000
313/313 [==============================] - 1s 860us/step - loss: 191.3925 - mean_squared_error: 191.3925
Epoch 2/1000
313/313 [==============================] - 0s 885us/step - loss: 47.9078 - mean_squared_error: 47.9078
Epoch 3/1000
313/313 [==============================] - 0s 902us/step - loss: 31.4849 - mean_squared_error: 31.4849
Epoch 4/1000
313/313 [==============================] - 0s 893us/step - loss: 22.8842 - mean_squared_error: 22.8842
Epoch 5/1000
313/313 [==============================] - 0s 861us/step - loss: 17.9222 - mean_squared_error: 17.9222
Epoch 6/1000
313/313 [==============================] - 0s 880us/step - loss: 14.1754 - mean_squared_error: 14.1754
Epoch 7/1000
313/313 [==============================] - 0s 866us/step - loss: 12.1417 - mean_squared_error: 12.1417
Epoch 8/1000
313/313 [==============================] - 0s 893us/step - loss: 10.0329 - mean_squared_error: 10.0329
Epoch 9/1000
313/313 [==============================] - 0s 873us/step 

In [39]:
for i in range(n_nonsimilar):
    similarity_exponents = model.get_layer('similarity_layer_' + str(i+1)).weights[0][0].numpy()
    print('Exponents multiplicating inside the function for ' + nonsimilar_keys[i] + ': ' + str(similarity_exponents))

multiplication_exponents = model.get_layer('multiplication_layer').weights[0][0].numpy()
print('Exponents multiplicating outside the function: ' + str(multiplication_exponents))

Exponents multiplicating inside the function for y+: [-0.99983215]
Exponents multiplicating outside the function: [0.99982715]
